#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [2]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 4.5 MB/s 
     |████████████████████████████████| 125 kB 14.9 MB/s 
     |████████████████████████████████| 512 kB 10.0 MB/s 
     |████████████████████████████████| 87 kB 3.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=809dc65ada32df76dfd5bf345fd5e72a0e92698dc8645042ddeea83f9aeacd1a
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire


Скачаем данные

In [3]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec  7 13:03 __MACOSX
drwxr-xr-x 1 root root 4096 Dec  5 14:37 sample_data


Посмотрим как выглядит один из файлов разметки

In [22]:
!cat data/train/apple_3.xml

<annotation>
	<folder>train</folder>
	<filename>apple_3.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1000</width>
		<height>708</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>584</xmin>
			<ymin>438</ymin>
			<xmax>867</xmax>
			<ymax>708</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>492</xmin>
			<ymin>141</ymin>
			<xmax>740</xmax>
			<ymax>394</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>176</xmin>
			<ymin>199</ymin>
			<xmax>490</xmax>
			<ymax>466</ymax>
		</bndbox>
	</object>
	<obj

## Релизуйте выборку для YoloV1 - 2 балла

In [23]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

In [24]:
image_paths = glob.glob(os.path.join('data/train/','*.jpg'))

In [25]:
fileptr = open("/content/data/train/orange_76.xml","r")

#read xml content from the file
xml_content= fileptr.read()

#change xml format to ordered dict
my_ordered_dict=xmltodict.parse(xml_content)
my_ordered_dict

{'annotation': {'folder': 'train',
  'filename': 'orange_76.jpg',
  'path': 'C:\\tensorflow1\\models\\research\\object_detection\\images\\train\\orange_76.jpg',
  'source': {'database': 'Unknown'},
  'size': {'width': '1620', 'height': '1080', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '494', 'ymin': '1', 'xmax': '1452', 'ymax': '741'}},
   {'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '822',
     'ymin': '643',
     'xmax': '1620',
     'ymax': '1066'}}]}}

In [26]:
len(my_ordered_dict['annotation']['object'])

2

In [27]:
dic_t = xmltodict.parse(open("/content/data/train/orange_76.xml","r").read())['annotation']['object']
dic_t[0]['bndbox']['xmax']

'1452'

In [28]:
im = np.array(Image.open('/content/data/train/mixed_8.jpg').convert("RGB"))
im.shape[0]

757

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [29]:
from sklearn import preprocessing

In [123]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.image_paths = glob.glob( os.path.join(data_dir,'*.jpg'))
        self.enc = preprocessing.LabelEncoder()
        self.box_paths = [x+".xml" for x in [os.path.splitext(x)[0] for x in self.image_paths]]
        assert len(self.image_paths) == len(self.box_paths)
        self.transforms = transforms
        for idx in range(len(self.box_paths)):
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
            _,classes =  self.__get_boxes_from_xml(self.box_paths[idx],image)
            if idx == 0:
              all_classes = classes
            classes = np.array(classes)
            all_classes = np.concatenate((all_classes,classes))
        all_classes = np.unique(all_classes)
        self.enc.fit(all_classes)


    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx],image)
        if self.transforms is not None:
          augs = self.transforms(image=image, bboxes = boxes, class_labels=class_labels)
          image = augs['image']
          boxes = augs['bboxes']
          class_labels = augs['class_labels']
        boxes, class_labels = torch.as_tensor(boxes, dtype=torch.float64),  torch.as_tensor(self.enc.transform(class_labels),dtype=torch.int8)
        return image.unsqueeze(dim=0), (boxes, class_labels)

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str,image):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      dic = xmltodict.parse(open(xml_filename,"r").read())['annotation']
      im_w = int(dic['size']['width'])
      if im_w == 0:
        im_w = image.shape[1]
      im_h = int(dic['size']['height'])
      if im_h == 0:
        im_h = image.shape[0]
      obj = dic['object']
      if isinstance(obj, dict):
        obj = [obj]
      for item in obj:
        boxes.append(self.__convert_to_yolo_box_params(
            [int(x) for x in [item['bndbox']['xmin'], item['bndbox']['ymin'], item['bndbox']['xmax'], item['bndbox']['ymax']]],
            im_w,im_h)
        ) 
        class_labels.append(item['name'])      
      return boxes,class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = [] 
      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)# x_center  
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)# width  
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height 
      return ans

In [124]:
transform = A.Compose([A.ToFloat (max_value=None, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()
                       ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
train_dataset = FruitDataset(transforms=transform,
    data_dir="./data/train")
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1)
test_dataset = FruitDataset(transforms=transform,
    data_dir="./data/test")
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1)

In [125]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        print(data)
        channels_sum += torch.mean(data[0], dim=[0,2,3])
        channels_squared_sum += torch.mean(data[0]**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [126]:
mean, std = get_mean_and_std(train_dataloader)
mean_tr,std_tr = mean.tolist(),std.tolist()
mean, std = get_mean_and_std(test_dataloader)
mean_test,std_test = mean.tolist(),std.tolist()

Выходные данные были обрезаны до нескольких последних строк (5000).
           [1., 1., 1.,  ..., 1., 1., 1.]],

          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]]])
tensor([[[[[0.1569, 0.1569, 0.1569,  ..., 0.1294, 0.1294, 0.1294],
           [0.1608, 0.1647, 0.1647,  ..., 0.1294, 0.1294, 0.1294],
           [0.1608, 0.1647, 0.1686,  ..., 0.1294, 0.1294, 0.1294],
           ...,
           [0.8667, 0.8588, 0.8549,  ..., 0.2549, 0.2784, 0.3255],
           [0.8196, 0.8314, 0.8627,  ..., 0.2392, 0.2235, 0.2353],
           [0.9098, 0.8784, 0.8510,  ..., 0.2471, 0.2196, 0.2196]],

          [[0.1216, 0.1216, 0.1216,  ..., 0.1020, 0.1020, 0.1020],
           [0.1255, 0.1294, 0.1294,  ..., 0.1020, 0.1020, 0.1020],
           [0.1255, 0.1294, 0.1333,  ..., 0.1020, 0.102

In [127]:
mean_tr

[0.8134564757347107, 0.6914448142051697, 0.5386324524879456]

In [128]:
std_tr

[0.27581095695495605, 0.3231784403324127, 0.4065982699394226]

In [129]:
mean_test

[0.8366212248802185, 0.7004530429840088, 0.5619865655899048]

In [130]:
std_test

[0.25586530566215515, 0.331956148147583, 0.40674322843551636]

In [131]:
WIDTH, HEIGHT = 448,448

train_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_tr, std=std_tr, max_pixel_value=255.0, always_apply=False, p=1.0),A.Resize(WIDTH, HEIGHT),
    A.pytorch.transforms.ToTensorV2(), ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
test_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_test, std=std_test, max_pixel_value=255.0, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()],
                            bbox_params=A.BboxParams(format='yolo',
                                                     label_fields=['class_labels']))

In [132]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[2], tuple)
assert len(train_dataset[2]) == 2
assert isinstance(train_dataset[2][0], torch.Tensor)
assert isinstance(train_dataset[2][1], tuple)
assert len(train_dataset[2][1]) == 2
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [133]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [134]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 16,
    num_workers = 4,
    shuffle=True, collate_fn=collate_fn)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=16,
    num_workers = 4, 
    shuffle=False,
    collate_fn=collate_fn
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [135]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

In [136]:
x = next(iter(train_dataloader))
print(x[1])

((tensor([[0.4724, 0.5706, 0.5490, 0.7765]], dtype=torch.float64), tensor([0], dtype=torch.int8)), (tensor([[0.2745, 0.8093, 0.2830, 0.3814],
        [0.3840, 0.3778, 0.2480, 0.3573],
        [0.6670, 0.4696, 0.3140, 0.3771],
        [0.5070, 0.1815, 0.2520, 0.3150],
        [0.2255, 0.2147, 0.2650, 0.3305]], dtype=torch.float64), tensor([0, 0, 0, 0, 0], dtype=torch.int8)), (tensor([[0.4971, 0.5988, 0.6714, 0.5525]], dtype=torch.float64), tensor([0], dtype=torch.int8)), (tensor([[0.5160, 0.5480, 0.6720, 0.6800]], dtype=torch.float64), tensor([0], dtype=torch.int8)), (tensor([[0.6789, 0.5133, 0.4273, 0.6401],
        [0.2958, 0.4441, 0.4450, 0.6436]], dtype=torch.float64), tensor([2, 2], dtype=torch.int8)), (tensor([[0.5818, 0.7111, 0.3693, 0.5111],
        [0.2076, 0.7111, 0.3673, 0.5222]], dtype=torch.float64), tensor([1, 1], dtype=torch.int8)), (tensor([[0.6970, 0.4813, 0.4660, 0.5129]], dtype=torch.float64), tensor([2], dtype=torch.int8)), (tensor([[0.5859, 0.2347, 0.6281, 0.1435],


Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [137]:
params = [ #[channels, Maxpool, kernel, padding, stride]
    [64,True,7,3,2],
    [192,True,3,1,1],
    [128,False,1,0,1],
    [256,False,3,1,1],
    [256,False,1,0,1],
    [512,True,3,1,1],
    (4,
    [[256,False,1,0,1],
    [512,False,3,1,1]]),
    [512,False,1,0,1],
    [1024,True,3,1,1],
    (2,
    [[512,False,1,0,1],
    [1024,False,3,1,1]]),
    [1024,False,3,1,1],
    [1024,False,3,1,2],
    (2,[[1024,False,3,1,1]])
]

In [138]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3,in_channels:int = 3, architecture:list=params,**kwargs):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C
        self.in_channels=in_channels
        self.arch = architecture
        self.net = self._create_conv_layers()
        self.fc = self._create_fcs(**kwargs)

    def forward(self, x):
        for block in self.net:
          x= block(x)
        return self.fc(x)


    def _create_conv_layers(self):
      in_channels = self.in_channels
      conv = nn.ModuleList()
      for block in self.arch:
        if isinstance(block, list):
          conv.append(CNNBlock(in_channels,*block[0:2],kernel_size=block[2],padding=block[3],stride=block[4]))
          in_channels = block[0]
        elif isinstance(block, tuple):
          for _ in range(block[0]):
            for small_block in block[1]:
              conv.append(CNNBlock(in_channels,*small_block[0:2],kernel_size=small_block[2],padding=small_block[3],stride=small_block[4]))
              in_channels=small_block[0]
        else:
          raise TypeError("architecture can include only types: 'list' or 'tuple'")
      return conv


    def _create_fcs(self,p = 0.1):
      return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * self.S * self.S, 4096),
            nn.Dropout(p = p),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, self.S * self.S * (self.C + self.B * 5)),
        )

temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

assert temp_model(train_dataset[0][0]).reshape(-1).shape[0] == expected_output_shape
temp_model(train_dataset[0][0]).size()

torch.Size([1, 637])

## Реализуйте YoloLoss - 3 балла

In [79]:
x = range(1,25)
print(x.reshape(-1,3,8))

AttributeError: ignored

In [81]:
print(*x[0:5],'|',*x[6:11])

1 2 3 4 5 | 7 8 9 10 11


In [89]:
x = torch.empty(2,2)
x[0][0],x[0][1],x[1][0],x[1][1] = 1,2,-1,-2
print(x)
print(x[0])
torch.max(x,dim=1)

tensor([[ 1.,  2.],
        [-1., -2.]])
tensor([1., 2.])


torch.return_types.max(
values=tensor([ 2., -1.]),
indices=tensor([1, 0]))

In [139]:
y = torch.tensor([[1,2],[3,4]])
z = torch.tensor([[-1,8],[-3,-4]])
yz = torch.cat([y.unsqueeze(0),z.unsqueeze(0)],dim=0)
print(yz)
torch.argmax(yz,dim=0)

tensor([[[ 1,  2],
         [ 3,  4]],

        [[-1,  8],
         [-3, -4]]])


tensor([[0, 1],
        [0, 0]])

In [58]:
from itertools import product

In [ ]:
def get_target(boxes, labels,S=7, B=2, C=3):
  label_matrix = torch.zeros((S, S, C + 5 * B))
  for box,i in enumerate(boxes):
    x,y,w,h = box.tolist()
    class_label = labels[i]
    class_label = int(class_label)

    i, j = int(S * y), int(S * x)
    x_cell, y_cell = S * x - j, S * y - i
    width_cell, height_cell = (w * S, h * S)

    # If no object already found for specific cell i,j
    # Note: This means we restrict to ONE object
    # per cell!
    if label_matrix[i, j, C] == 0:
        # Set that there exists an object
        label_matrix[i, j, C] = 1

        # Box coordinates
        box_coordinates = torch.tensor(
            [x_cell, y_cell, width_cell, height_cell]
        )

        label_matrix[i, j, C+1:C+5] = box_coordinates

        # Set one hot encoding for class_label
        label_matrix[i, j, class_label] = 1

        return label_matrix

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        self.pred = predictions.reshape(self.B,self.S,self.S,self.C+5)
        iou = torch.empty((self.S,self.S,self.C+self.B*5))
        box_pred = torch.empty((self.S,self.S))
        for x,y,n in product(range(self.S),range(self.S),range(self.B)):
          iou[self.B,self.S,self.S] = intersection_over_union(self.pred[x,y,n,0:4],self.target[...,self.C+1:self.C+5])
        best_boxes = torch.argmax(iou,dim=0)
        I_box = target[...,self.C]
        for x,y in product(range(self.S),range(self.S)):
          box_pred[x,y]=self.pred[x,y,int(best_boxes[x,y]),0:4]
        box_pred = I_box*box_pred
        box_targ = I_box * target[..., self.C+1:self.C+5]
        pass

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
def non_max_suppression(bboxes, iou_threshold, threshold):
    ## YOUR CODE
    pass

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model = # YOUR CODE
n_epochs = # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image